In [1]:
import random
import mesa
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

class Sheep(mesa.Agent):
    def __init__(self, unique_id, pos, model):
        super().__init__(unique_id, model)
        self.pos = pos

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def step(self):
        self.move()

class Wolf(mesa.Agent):
    def __init__(self, unique_id, pos, model):
        super().__init__(unique_id, model)
        self.pos = pos

    def move(self):
        sheep_agents = self.model.schedule.agents_by_type[Sheep]
        if len(sheep_agents) > 0:
            # Find the closest sheep
            closest_sheep = min(sheep_agents, key=lambda a: self.distance(a.pos))
            # Move towards the sheep
            dx = 1 if closest_sheep.pos[0] > self.pos[0] else -1
            dy = 1 if closest_sheep.pos[1] > self.pos[1] else -1
            new_position = (self.pos[0] + dx, self.pos[1] + dy)
        else:
            # If there are no sheep, move randomly
            possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
            new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def distance(self, pos):
        return abs(self.pos[0] - pos[0]) + abs(self.pos[1] - pos[1])

    def step(self):
        self.move()

class WolfPredationModel(mesa.Model):
    def __init__(self, num_sheep, num_wolves, width, height):
        self.num_sheep = num_sheep
        self.num_wolves = num_wolves
        self.grid = MultiGrid(width, height, torus=True)
        self.schedule = RandomActivation(self)
        self.running = True

        # Create sheep agents
        for i in range(self.num_sheep):
            pos = (random.randrange(width), random.randrange(height))
            sheep = Sheep(i, pos, self)
            self.grid.place_agent(sheep, pos)
            self.schedule.add(sheep)

        # Create wolf agents
        for i in range(self.num_wolves):
            pos = (random.randrange(width), random.randrange(height))
            wolf = Wolf(i, pos, self)
            self.grid.place_agent(wolf, pos)
            self.schedule.add(wolf)

        # Collect data
        self.datacollector = DataCollector(
            {"Sheep": lambda m: m.schedule.get_breed_count(Sheep),
             "Wolves": lambda m: m.schedule.get_breed_count(Wolf)}
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()


In [2]:
import random

from mesa import Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

from agents import Sheep, Wolf


class WolfPredationModel(Model):
    """
    Wolf-sheep predation model.
    """

    def __init__(self, height, width, sheep_count, wolf_count):
        """
        Create a new Wolf-Sheep model with the given parameters.

        Args:
            height: Height of the grid.
            width: Width of the grid.
            sheep_count: Number of initial sheep.
            wolf_count: Number of initial wolves.
        """
        super().__init__()
        self.height = height
        self.width = width

        # Create a multi-grid to store the agents
        self.grid = MultiGrid(self.height, self.width, torus=True)

        # Create a scheduler to handle agent activation order
        self.schedule = RandomActivation(self)

        # Create data collectors to track population and agent variables
        self.datacollector = DataCollector(
            {"Sheep": lambda m: m.schedule.get_breed_count(Sheep),
             "Wolves": lambda m: m.schedule.get_breed_count(Wolf)})

        # Create sheep agents
        for i in range(sheep_count):
            x = random.randrange(self.width)
            y = random.randrange(self.height)
            sheep = Sheep(self.next_id(), (x, y), self)
            self.grid.place_agent(sheep, (x, y))
            self.schedule.add(sheep)

        # Create wolf agents
        for i in range(wolf_count):
            x = random.randrange(self.width)
            y = random.randrange(self.height)
            wolf = Wolf(self.next_id(), (x, y), self)
            self.grid.place_agent(wolf, (x, y))
            self.schedule.add(wolf)

        self.running = True

    def step(self):
        """
        Advance the model by one step.
        """
        self.schedule.step()
        self.datacollector.collect(self)

    def run_model(self, steps):
        """
        Run the model for the given number of steps.

        Args:
            steps: Number of steps to run the model.
        """
        for i in range(steps):
            self.step()


In [3]:
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.modules import CanvasGrid, ChartModule, TextElement
from mesa.visualization.UserParam import UserSettableParameter

from agents import Sheep, Wolf
from model import WolfPredationModel


def wolf_sheep_portrayal(agent):
    if agent is None:
        return

    portrayal = {}

    if type(agent) is Sheep:
        portrayal["Shape"] = "resources/sheep.png"
        portrayal["scale"] = 0.9
        portrayal["Layer"] = 1

    elif type(agent) is Wolf:
        portrayal["Shape"] = "resources/wolf.png"
        portrayal["scale"] = 0.8
        portrayal["Layer"] = 2
        portrayal["text"] = str(agent.energy)

    portrayal["x"] = agent.pos[0]
    portrayal["y"] = agent.pos[1]
    portrayal["text_color"] = "white"

    return portrayal


canvas_element = CanvasGrid(wolf_sheep_portrayal, 20, 20, 500, 500)
chart_element = ChartModule([{"Label": "Wolves", "Color": "#AA0000"},
                             {"Label": "Sheep", "Color": "#666666"}])

class PopulationText(TextElement):
    def __init__(self):
        pass

    def render(self, model):
        sheep_count = len([a for a in model.schedule.agents if type(a) is Sheep])
        wolf_count = len([a for a in model.schedule.agents if type(a) is Wolf])
        return f"Sheep count: {sheep_count}, Wolf count: {wolf_count}"

model_params = {
    "height": 20,
    "width": 20,
    "initial_sheep": UserSettableParameter("slider", "Initial sheep count", 100, 10, 500, 10),
    "sheep_reproduce": UserSettableParameter("slider", "Sheep reproduction rate", 0.08, 0.01, 1.0, 0.01),
    "initial_wolves": UserSettableParameter("slider", "Initial wolf count", 50, 10, 500, 10),
    "wolf_reproduce": UserSettableParameter("slider", "Wolf reproduction rate", 0.05, 0.01, 1.0, 0.01),
    "wolf_gain_from_food": UserSettableParameter("slider", "Wolf gain from food", 20, 1, 100, 1),
    "sheep_gain_from_food": UserSettableParameter("slider", "Sheep gain from food", 4, 1, 10, 1),
}



server = ModularServer(WolfPredationModel, [canvas_element, chart_element, PopulationText()], "Wolf Sheep Predation Model", model_params)

server.port = 8521
server.launch()


C:\Users\PrakrutUpadhyay\anaconda3\lib\site-packages\mesa\visualization\UserParam.py:69: UserWarning: UserSettableParameter is deprecated in favor of UserParam objects such as Slider, Checkbox, Choice, StaticText, NumberInput. See the examples folder for how to use them. UserSettableParameter will be removed in the next major release.
  warn(


AttributeError: 'WolfPredationModel' object has no attribute 'datacollector'